<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Recomendação Trending

<!-- Este notebook contém 2 exemplos de recomendação Top N:

- **Top N consumidos**: os N itens mais consumidos pelos usuários
- **Top N avaliados**: os N itens melhores avaliados pelos usuários -->

O dataset a ser utilizado será o [MovieLens](https://grouplens.org/datasets/movielens/), cuja análise exploratória foi feita no exemplo prático do módulo **Introdução aos Sistemas de Recomendação**.

In [1]:
import os
import re
import sys
import pandas as pd
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

# Carregando e processando o dataset

Para mais informações desta sessão, consulte o notebook `Introdução aos Sistemas de Recomendação` do módulo 01.

## Arquivo de avaliações

Upload file `ratings.parquet`

In [2]:
%%time
_ = files.upload()

Saving ratings.parquet to ratings.parquet
CPU times: user 842 ms, sys: 205 ms, total: 1.05 s
Wall time: 1min 7s


In [3]:
def convert_timestamp_to_date(timestamp:int):
    return datetime.fromtimestamp(timestamp).date()

df_ratings = pd.read_parquet('ratings.parquet')
df_ratings['date'] = df_ratings['timestamp'].apply(convert_timestamp_to_date)
df_ratings.tail()

,user_id,item_id,rating,timestamp,date
1000204,6040,1091,1,956716541,2000-04-26
1000205,6040,1094,5,956704887,2000-04-25
1000206,6040,562,5,956704746,2000-04-25
1000207,6040,1096,4,956715648,2000-04-26
1000208,6040,1097,4,956715569,2000-04-26


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [4]:
%%time
_ = files.upload()

Saving movies.parquet to movies.parquet
CPU times: user 631 ms, sys: 91 ms, total: 722 ms
Wall time: 57.9 s


In [5]:
def extract_year_from_title(title:str, regex='(\d{4})'):
    match = re.search(regex, title)
    return None if match is None else match.group()

def convert_genres_to_list(genres:str, separator='|'):
    return genres.split(separator)

df_items = pd.read_parquet('movies.parquet')
df_items['genres'] = df_items['genres'].apply(convert_genres_to_list)
df_items['year'] = df_items['title'].apply(extract_year_from_title)
df_items.tail()

,item_id,title,genres,year
3878,3948,Meet the Parents (2000),[Comedy],2000
3879,3949,Requiem for a Dream (2000),[Drama],2000
3880,3950,Tigerland (2000),[Drama],2000
3881,3951,Two Family House (2000),[Drama],2000
3882,3952,"Contender, The (2000)","[Drama, Thriller]",2000


# Cálculo do Trending

A recomendação _trending_ busca apresentar os itens que tiveram maior _lift_ de consumo em uma determinada janela de tempo. Em termos matemáticos, o _lift_ pode ser definido como:

$$lift = \frac{consumoJanelaAtual-consumoJanelaAnterior}{consumoJanelaAnterior}$$

Neste notebook, utilizaremos como janela de tempo o mês de consumo do item. A função abaixo nos auxilia a definir uma coluna `window` que será utilizada na lógica do algoritmo.

In [7]:
def extract_year_month(date):
    return '{:04d}-{:02d}'.format(date.year, date.month)

# Vamos usar como janela o ano e mês de uma avaliação
df_ratings['window'] = df_ratings['date'].apply(extract_year_month)
df_ratings.tail()

,user_id,item_id,rating,timestamp,date,window
1000204,6040,1091,1,956716541,2000-04-26,2000-04
1000205,6040,1094,5,956704887,2000-04-25,2000-04
1000206,6040,562,5,956704746,2000-04-25,2000-04
1000207,6040,1096,4,956715648,2000-04-26,2000-04
1000208,6040,1097,4,956715569,2000-04-26,2000-04


## Consumo por janela temporal

In [16]:
df_window_consumptions = (
        df_ratings
        .groupby(['item_id', 'window'])
        .agg({'user_id':'count'})
        .reset_index()
        .rename({'user_id': 'count'}, axis=1)
        .sort_values(by=['item_id', 'window'])
)

df_window_consumptions

,item_id,window,count
0,1,2000-04,17
1,1,2000-05,165
2,1,2000-06,128
3,1,2000-07,203
4,1,2000-08,386
...,...,...,...
65635,3952,2002-09,1
65636,3952,2002-11,1
65637,3952,2002-12,3
65638,3952,2003-01,2


## Shift temporal

Para realizar operações entre o valor atual e o anterior de uma janela temporal podemos **deslocar** (_shift_) os valores de uma coluna a partir de um agrupamento. Exemplo:

| Agrupamento | Valor | Shift Valor |
|--------|-------|-------------|
| A      | A1    | N/A         |
| A      | A2    | A1          |
| B      | B1    | N/A         |
| B      | B2    | B1          |
| B      | B3    | B2          |

In [22]:
df_window_consumptions.sort_values(by=['item_id','window'], inplace=True)

# shift é quantidade de casas que irão ser mandadas para baixo
df_window_consumptions['count_previous'] = (
      df_window_consumptions
      .groupby(['item_id'])['count']
      .shift(1)

)

df_window_consumptions

,item_id,window,count,count_previous
0,1,2000-04,17,NaN
1,1,2000-05,165,17.0
2,1,2000-06,128,165.0
3,1,2000-07,203,128.0
4,1,2000-08,386,203.0
...,...,...,...,...
65635,3952,2002-09,1,2.0
65636,3952,2002-11,1,1.0
65637,3952,2002-12,3,1.0
65638,3952,2003-01,2,3.0


## Lift

Implementando a seguinte fórmula:
$$lift = \frac{contagemJanelaAtual-contagemJanelaAnterior}{contagemJanelaAnterior}$$

In [23]:
df_window_consumptions['lift'] = (df_window_consumptions['count'] - df_window_consumptions['count_previous']) / df_window_consumptions['count_previous']
df_window_consumptions

,item_id,window,count,count_previous,lift
0,1,2000-04,17,NaN,NaN
1,1,2000-05,165,17.0,8.705882
2,1,2000-06,128,165.0,-0.224242
3,1,2000-07,203,128.0,0.585938
4,1,2000-08,386,203.0,0.901478
...,...,...,...,...,...
65635,3952,2002-09,1,2.0,-0.500000
65636,3952,2002-11,1,1.0,0.000000
65637,3952,2002-12,3,1.0,2.000000
65638,3952,2003-01,2,3.0,-0.333333


## Especificando janela de recomendação

Para a recomendação precisamos de uma janela de referência. Por exemplo:

- Trending na janela atual?
- Trending na janela anterior?
- Trending em uma janela específica?

Definida a janela de referência, utilizamos o valor do _lift_ como _score_ do item e ordenamos pelo _score_.

In [24]:
prediction_window = '2003-01'
(
    df_window_consumptions
    .query('window == @prediction_window')
    .rename({'lift': 'score'}, axis=1)
    .sort_values(by='score')
)

,item_id,window,count,count_previous,score
18836,1179,2003-01,1,5.0,-0.80
21114,1266,2003-01,1,5.0,-0.80
18426,1136,2003-01,1,5.0,-0.80
36667,2193,2003-01,1,5.0,-0.80
59667,3549,2003-01,1,4.0,-0.75
...,...,...,...,...,...
59977,3566,2003-01,5,1.0,4.00
782,32,2003-01,6,1.0,5.00
64914,3897,2003-01,6,1.0,5.00
32959,2011,2003-01,7,1.0,6.00
